# Set up

In [33]:
# update path with data dir
import sys
sys.path.append('../data/')

In [34]:
import player_data as player
import pandas as pd
from datetime import datetime
import patsy

# Get data

In [68]:
file_name = 'csvs/element_gameweek_' + datetime.today().strftime('%Y-%m-%d') + '.csv'

In [69]:
try:
    element_gameweek_df = pd.read_csv(file_name)
except:
    element_gameweek_df = player.get_element_gameweek_df()
    element_gameweek_df.to_csv(file_name)

In [70]:
current_event = 30
element_types = [1, 2, 3, 4]
threshold_minutes = 1

In [72]:
element_minute_df = element_gameweek_df.groupby('element', as_index=False)['minutes'].sum()
elements = element_minute_df[element_minute_df['minutes'] >= threshold_minutes]['element'].values

fixtures_df = player.get_fixtures_df()
element_df = player.get_element_df(['id', 'element_type', 'team', 'web_name'])
element_df = element_df[element_df['id'].isin(elements)]

element_df['element'] = element_df['id']

element_df = element_df[['element_type', 'element', 'team']]

gameweek_fixtures_df = fixtures_df[
    fixtures_df['event'] == 34
]

home_gameweek_fixtures_df = gameweek_fixtures_df.copy()
home_gameweek_fixtures_df['team'] = home_gameweek_fixtures_df['team_h']
home_gameweek_fixtures_df['was_home'] = 1


away_gameweek_fixtures_df = gameweek_fixtures_df.copy()
away_gameweek_fixtures_df['team'] = home_gameweek_fixtures_df['team_a']
away_gameweek_fixtures_df['was_home'] = 0

gameweek_fixtures_df = pd.concat([home_gameweek_fixtures_df, away_gameweek_fixtures_df])

gameweek_fixtures_df['opposition_team'] = \
gameweek_fixtures_df.apply(player.calculate_opposition_team, axis=1)

gameweek_fixtures_df['own_team'] = \
gameweek_fixtures_df.apply(player.calculate_own_team, axis=1)

gameweek_fixtures_df = gameweek_fixtures_df.join(
    element_df.set_index('team'), on='team')[[
    'own_team', 'opposition_team', 'element', 'was_home', 'element_type']]

In [63]:
f = 'was_home ~ was_home + C(element) + C(opposition_team)'

In [66]:
_, gameweek_fixtures_explanatory_df = \
patsy.dmatrices(f, gameweek_fixtures_df, return_type='dataframe')

In [67]:
gameweek_fixtures_explanatory_df.head()

,Intercept,C(element)[T.2],C(element)[T.3],C(element)[T.4],C(element)[T.5],C(element)[T.6],C(element)[T.7],C(element)[T.8],C(element)[T.9],C(element)[T.10],...,C(opposition_team)[T.12],C(opposition_team)[T.13],C(opposition_team)[T.14],C(opposition_team)[T.15],C(opposition_team)[T.16],C(opposition_team)[T.17],C(opposition_team)[T.18],C(opposition_team)[T.19],C(opposition_team)[T.20],was_home
325,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
325,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
325,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
325,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
325,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
